In [1]:
import numpy as np
import torch
import torch.nn.functional as F
from torch.autograd.functional import hessian, jacobian

In [2]:
x = np.linspace(0,1,10, dtype=np.float)
t = np.linspace(0,1,10, dtype=np.float)
x, t = np.meshgrid(x, t) 
x = x.flatten()
t = t.flatten()
X = np.concatenate((x.reshape(-1,1), t.reshape(-1, 1)), axis=1)

num_points, input_dim = X.shape
hidden_neurons = 100
output_dim = 1

In [3]:
def g_bc(x, t):
    return (1 - t) * torch.sin(np.pi * x)

def g_nn(x, t, N):
    return x * (1 - x) * t * N

def g_trial(val, N):
    x, t = val.T
    return g_bc(x, t) + g_nn(x, t, N)

def ode_loss(x, N):
    #dudx2 = hessian(g_trial, (x, N))
    #dudt = jacobian(g_trial, (x, N))
    
    loss_all = torch.zeros(x.size(0))
    
    # Split into each x_i, N_i pair before computing gradient!
    for i in range(x.size(0)):
        #dudx2 = hessian(g_trial, (x[i,:], N[i]))[0][0][0][0]
        J_g, _ = jacobian(g_trial, (x[i,:], N[i]))
        H_g, _ = hessian(g_trial, (x[i,:], N[i]))
        
        #print(J_g[0,1])
        #print(H_g[0][0,0])
        
        #print("\n\ndudx = ", dudx2,"\ndudt=", dudt)
        loss_all[i] = (J_g[0,1] - H_g[0][0,0])**2
        
    #print("loss=",loss)
    #return torch.sum(g_trial(x, N))
    return torch.sum(loss_all) / x.size(0)
# https://pytorch.org/docs/stable/autograd.html#torch.autograd.functional.hessian
#u_hess = hessian(g_trial, (x, N))
# https://pytorch.org/docs/stable/autograd.html#torch.autograd.functional.hessian
#u_jacob = jacobian(g_trial, (x, N))

torch.autograd.set_detect_anomaly(True)

In [4]:
dtype = torch.float
#device = torch.device("cuda:0")
device = torch.device("cpu")

x = torch.from_numpy(X)
x = x.to(dtype)
x.requires_grad = True

# Create random Tensors for weights.
w1 = torch.randn(input_dim, hidden_neurons, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(hidden_neurons, output_dim, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-7
for t in range(5):
    N_output = F.relu(x.mm(w1)).mm(w2)
    
    # Compute and print loss
    loss = ode_loss(x, N_output)
    print(loss)
    # Use autograd to compute the backward pass.
    loss.backward()
    # Update weights using gradient descent
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        # Manually zero the gradients after updating weights
        w1.grad.zero_()
        w2.grad.zero_()

tensor(28.1720, grad_fn=<DivBackward0>)


/home/nick/anaconda3/lib/python3.8/site-packages/torch/autograd/__init__.py:130: UserWarning: Error detected in torch::autograd::AccumulateGrad. Traceback of forward call that caused the error:
  File "/home/nick/anaconda3/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/nick/anaconda3/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/home/nick/anaconda3/lib/python3.8/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/nick/anaconda3/lib/python3.8/site-packages/traitlets/config/application.py", line 845, in launch_instance
    app.start()
  File "/home/nick/anaconda3/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 612, in start
    self.io_loop.start()
  File "/home/nick/anaconda3/lib/python3.8/site-packages/tornado/platform/asyncio.py", line 199, in start
    self.asyncio_loop.run_forever()
  File "/home/nick/anaconda3/lib

RuntimeError: leaf variable has been moved into the graph interior